# Modules and Packages
__Modules__: Normally Module is a single Python source file. Its a basic tool for organizing code. Represented by module object.It is loaded with import statement. 


__Packages__: Packages in Python is just a special type of module.It can contain other modules including other packages.
So packages are a way to define hierarchies of modules in Python. This allows to group modules with similar functionality together in a ways that express their cohesiveness.Packages provide high level structure to Python code.

Packages are generally represented by directories in the file system while modules are represented by single files.

In [1]:
import urllib
import urllib.request

In [2]:
type(urllib)

module

In [3]:
type(urllib.request)

module

In the above case, request is nested inside urllib. Hence urllib is a package and request is a module.Here only urllib is bound to a name, so it is not possible to access request sub-module directly.When imported this way access to the sub-module via a fully qualified hierarchical module name.

In [4]:
# import a sub-module directly using the from import syntax.
# This only binds request to a name in the local namespace. The parent urllib package will have been imported
# but won't be directly accessible via the urllib name.
from urllib import request
request # The sub-module knows its own hierarchical module name

<module 'urllib.request' from '/usr/lib/python3.7/urllib/request.py'>

In [5]:
# __path__ attribute is a list of file system paths indicating where urllib searches to find nested modules.
# This hints at the nature of the distinction between packages and modules.
# In Python3 prior to version 3.3, __path__ was just a single string. Not a list.
urllib.__path__

['/usr/lib/python3.7/urllib']

In [21]:
urllib.request.__path__

AttributeError: module 'urllib.request' has no attribute '__path__'

# Locating Modules
When python encounter import statement, it looks on the file system for the corresponding Python source file
and loads that code. But how does Python knows where to look? Python checks the path attribute of the standard
sys module, commonly referred to as sys.path.This is a list of directories.Python checks each entry in sys.path
object in order until it finds the appropriate Python source file i.e module.In case there is no match in all
directories in sys.path, ImportError is raised.

This can be modified by adding new path to sys.path using append() method as sys.path merely a list object.
Sometimes this is the best way to make code available to Python.There is an another way to add entries in sys.path, that does not require direct manipulation of sys.path.
The PYTHONPATH environment variable is a list of paths that are added to sys.path when Python starts.The format of
PYTHONPATH is same as PATH on the platform. In windows its a ; separated list of directories and in linux/mac
its a : separated list of directories.

In [7]:
import sys
sys.path

['/media/hdd/atelier/Workspace/PycharmProjects/pystart/OrganizingProgramms',
 '/media/hdd/atelier/Workspace/PycharmProjects',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/home/mithun/.ipython']

# Creating packages
To create a module, create a python source file in a directory contained in sys.path. Creating package is not much different.Package can be created by following below ways -

1. First create a package's root directory. This root directory needs to be in some directory on sys.path.
2. Create a file called __init__.py in that root directory.This is called package init file.This makes a package a module.__init__.py can be and often is empty.Its presence alone suffices to establish the package.

Since PEP420 which was introduced in Python 3.3, __init__.py file is not technically required for packages any more.This is actually required for earlier version of python 3.3.
Furthermore __init__.py provides powerful tool for package initialization.But it is recommended to include __init__.py file when possible, because explicit is better than implicit.The existance of package initialization
file is an unambiguous signal that a Python developer intend for a direcory to be a package.

In [8]:
%%bash
mkdir -p demo_reader
touch demo_reader/__init__.py

In [9]:
%%writefile demo_reader/__init__.py
print("demo_reader is being imported!")

Overwriting demo_reader/__init__.py


In [10]:
import demo_reader # Successfully imported and executed __init__.py file

demo_reader is being imported!


In [11]:
# demo_reader is a module even though on the file system the name demo_reader refers to a directory.
# A package is a directory containing __init__.py file
demo_reader.__file__

'/media/hdd/atelier/Workspace/PycharmProjects/pystart/OrganizingProgramms/demo_reader/__init__.py'

# Working with package and subpackage
Lets create a python source file called multireader.py inside demo_reader.
change the content of demo_reader/__init__.py file as "# demo_reader/__init__.py".

In [12]:
%%writefile demo_reader/__init__.py
# demo_reader/__init__.py

Overwriting demo_reader/__init__.py


In [13]:
import demo_reader.multireader
rd = demo_reader.multireader.MultiReader('demo_reader/__init__.py')
print(rd.read())
rd.close()

# demo_reader/__init__.py



In [14]:
%%bash
mkdir -p demo_reader/compressed
touch demo_reader/compressed/__init__.py

Added two more python source files gzipped.py and bzipped.py inside demo_reader/compressed subpackage.Lets import them.

In [15]:
import demo_reader
import demo_reader.multireader
import demo_reader.compressed
import demo_reader.compressed.gzipped
import demo_reader.compressed.bzipped

In [23]:
%%bash
# Creating one sample file with bz2 compression
python -m demo_reader.compressed.bzipped sample.bz2 data compressed with bz2

In [22]:
%%bash
# Creating one sample file with gz compression
python -m demo_reader.compressed.gzipped sample.gz data compressed with gz

In [18]:
# Let's read the files
from demo_reader.multireader import MultiReader
bz = MultiReader('sample.bz2')
gz = MultiReader('sample.gz')
print(bz.read())
print(gz.read())
bz.close()
gz.close()

data compressed with bz2
data compressed with gz


# Relative imports
There are two ways the modules can be imported.
1. Absolute import - specify all of the ancestor modules of any module you want to import  
Example - import demo_reader.compressed.gzipped  
from demo_reader.compressed import gzipped

1. Relative import - mention equal number of dots(.) with the number of ancestor packages before the module name
that is doing the import.Instead of specifying imports with absolute paths from the root of package tree, it is specified relative to the importing module.Hence relative imports.It can only be used with the "from module import names" form of import.Relative imports also works only to import modules within the current top level package.Below are some of the benefits of relative imports.
- Reduces typing in deeply nested package structure
- Promote a certain form of modifiability since they allow you a principle, to rename top-level and subpackages in
some cases.  
Example - from . import gzipped  
import .gzipped - Will raise ImportError

It is advisable to use absolute import.

In [19]:
%%bash
mkdir -p demo_reader/util
touch demo_reader/util/__init__.py

Created one more subpackage util inside demo_reader and added __init__.py file.

# \__all\__ attribute
This is a module level attribute.This lets you control which attributes are imported when from module import * form of import is being used.If \__all\__ is not specified then from module import * imports all public names those without leading underscores from the imported module.This module attribute must be a list of strings and each string indicates a name which will be imported when the * syntax is used. 

In [ ]:
from pprint import pprint
pprint(locals()) # baseline display of the local variables